In [15]:
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay

def calculateDates(small_window, window, start_date):
    # create a start date
    start_date = pd.Timestamp(start_date)

    # add end date
    end_date = start_date + BDay(window)

    # include a history window for first day
    start_date = start_date - BDay(small_window)

    return {'start_date':start_date,'end_date':end_date}

- todo test format of date


In [16]:
import csv
import glob
import numpy as np
import pandas as pd

dates = calculateDates(small_window = 10, window = 1000, start_date = '2018-01-17')

# list the files to be read
path = "./data/full_history/*.csv"
all_files = np.array(glob.glob(path))

# create an empty list to add each stocks data frame
li=[]

for filename in all_files[1:100]:
    # read om the file
    df = pd.read_csv(filename, parse_dates=[0])
    
    # drop all extra columns
    df = df.drop(labels=list(['volume','close','high','low','adjclose']), axis='columns')
    
    # rename the stock value column after the stock
    stock_name = filename.split(sep='/')[3].split(sep='.')[0].lower()
    df = df.rename(columns={'open': stock_name})
    
    # select only relevant date
    df = df.loc[df['date'] > start_date]    
    
    # if the stock has no data after the start date, do not add it
    if df.empty:
        continue
        
    df = df[:window]  
    
    # set index as date in order to join
    df=df.set_index('date')
    
    # add to the list
    li.append(df)

# combine all the data frames into one large
df = pd.concat(li, join='outer', axis=1)

df

,isd,men,dfnl,srf,cnacu,hud,intg,fun,iusb,evf,...,wynd,otex,eigr,ebtc,ent,mhn,wneb,pkg,east,arct
date,,,,,,,,,,,,,,,,,,,,,
2018-01-18,14.88,11.24,24.865000,10.00,NaN,NaN,24.150000,64.959999,50.580002,6.49,...,54.266365,33.680000,8.60,35.410000,2.06,13.30,11.10,126.599998,4.050,9.55
2018-01-19,14.79,11.27,24.860001,9.89,NaN,NaN,23.170000,65.309998,50.560001,6.44,...,56.144470,33.750000,9.25,34.560001,2.05,13.30,10.90,126.690002,4.180,9.30
2018-01-22,14.88,11.29,24.879999,9.94,NaN,NaN,23.520000,65.930000,50.490002,6.50,...,57.155758,33.580002,9.00,34.090000,2.00,13.25,11.10,127.800003,4.170,9.36
2018-01-23,14.86,11.26,25.240000,10.00,NaN,NaN,23.150000,66.239998,50.520000,6.51,...,57.200901,33.810001,8.85,34.860001,2.11,13.28,11.00,126.809998,4.370,9.37
2018-01-24,14.94,11.25,25.370001,9.79,NaN,NaN,23.049999,67.250000,50.520000,6.58,...,57.665913,34.209999,9.00,35.500000,2.26,13.27,11.05,126.989998,4.122,9.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-29,13.58,9.72,22.740000,8.10,NaN,21.110001,30.650000,49.599998,48.740002,6.30,...,34.950001,33.669998,11.47,31.910000,2.36,11.78,10.18,91.410004,7.620,4.85
2018-10-30,13.54,9.65,22.379999,7.91,NaN,20.309999,30.650000,52.549999,48.669998,6.22,...,34.139999,32.720001,11.36,31.100000,2.34,11.74,10.16,89.570000,7.580,5.13
2018-10-31,13.56,9.67,22.905001,8.03,NaN,21.110001,30.650000,51.950001,48.599998,6.20,...,35.299999,33.669998,11.93,31.990000,2.42,11.69,10.21,93.839996,7.577,5.25


False